In [1]:
import pickle
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.base import clone
from sklearn.tree import DecisionTreeClassifier

In [2]:
import pandas as pd
import numpy as np
import os

In [66]:
pd.set_option('display.max_columns', None)

In [10]:
data=pd.read_csv("ObesityDataSet.csv")

In [11]:
data.head()

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,Female,21.0,1.62,64.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,0.0,1.0,no,Public_Transportation,Normal_Weight
1,Female,21.0,1.52,56.0,yes,no,3.0,3.0,Sometimes,yes,3.0,yes,3.0,0.0,Sometimes,Public_Transportation,Normal_Weight
2,Male,23.0,1.80,77.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,2.0,1.0,Frequently,Public_Transportation,Normal_Weight
3,Male,27.0,1.80,87.0,no,no,3.0,3.0,Sometimes,no,2.0,no,2.0,0.0,Frequently,Walking,Overweight_Level_I
4,Male,22.0,1.78,89.8,no,no,2.0,1.0,Sometimes,no,2.0,no,0.0,0.0,Sometimes,Public_Transportation,Overweight_Level_II


In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2111 entries, 0 to 2110
Data columns (total 17 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Gender                          2111 non-null   object 
 1   Age                             2111 non-null   float64
 2   Height                          2111 non-null   float64
 3   Weight                          2111 non-null   float64
 4   family_history_with_overweight  2111 non-null   object 
 5   FAVC                            2111 non-null   object 
 6   FCVC                            2111 non-null   float64
 7   NCP                             2111 non-null   float64
 8   CAEC                            2111 non-null   object 
 9   SMOKE                           2111 non-null   object 
 10  CH2O                            2111 non-null   float64
 11  SCC                             2111 non-null   object 
 12  FAF                             21

In [31]:
data.describe()

,Age,Height,Weight,FCVC,NCP,CH2O,FAF,TUE
count,2111.000000,2111.000000,2111.000000,2111.000000,2111.000000,2111.000000,2111.000000,2111.000000
mean,24.312600,1.701677,86.586058,2.419043,2.685628,2.008011,1.010298,0.657866
std,6.345968,0.093305,26.191172,0.533927,0.778039,0.612953,0.850592,0.608927
min,14.000000,1.450000,39.000000,1.000000,1.000000,1.000000,0.000000,0.000000
25%,19.947192,1.630000,65.473343,2.000000,2.658738,1.584812,0.124505,0.000000
50%,22.777890,1.700499,83.000000,2.385502,3.000000,2.000000,1.000000,0.625350
75%,26.000000,1.768464,107.430682,3.000000,3.000000,2.477420,1.666678,1.000000
max,61.000000,1.980000,173.000000,3.000000,4.000000,3.000000,3.000000,2.000000


In [14]:
data["NObeyesdad"].value_counts()/len(data)*100

Obesity_Type_I         16.627191
Obesity_Type_III       15.348176
Obesity_Type_II        14.069162
Overweight_Level_II    13.737565
Overweight_Level_I     13.737565
Normal_Weight          13.595452
Insufficient_Weight    12.884889
Name: NObeyesdad, dtype: float64

### Descripción de columnas utiles

Basados en el analisis de la data que se realizó, se puede observar que el campo de edad presenta rangos utiles que pueden ser combinados con otros campos para mejorar la performance del modelo. El campo de family_history_with_overweight presenta una correlacion de 0.51 con el target y en el analisis se demostró que para un BMI mayor a 28 (sobrepeso-obesidad) hay una tendencia a tener familiares con sobrepeso.

Por otro lado, BMI es un indicador que se elaboró que demostro una correlacion muy alta con los niveles de obesidad y tiene rangos bien delimitados por categorias. Esto se puede apreciar visualmente en el analisis de la data. Por esta razon, no se realizara algun binning o trabajo con esta columna.

Otros campos numericos con los que se podria realizar alguna combinacion son : Frequent consumption of high caloric food (FAVC) y Physical activity frequency (FAF). Estos campos tienen una buena correlacion connel target y adicionalmente de manera empirica podemos afirmar que son factores importantes para poder predecir el nivel de obesidad. En el analisis de la data FAVC demostro un pico en 1.00 en donde todas las categorias se reunen con diferentes intensidades, por lo que no se puede aprovechar  de hacer algun tipo de binning en esta columna. Por otro lado, FAF demostro tener unos rangos utiles cercanos al 1,2 y 3.


### Proposito del Feature Engineering

Este dataset no tiene datos nulos o categorias con un valor desconocido. El target esta bien distribuido por las categorias. Por lo tanto, el siguiente paso seria cambiar las columnas ordinales por numeros que demuestren la jerarquia de las opciones y las columnas binarias por valores de 0 y 1. Por ultimo, transformar a One Hot encoding las columnas categoricas. En este caso solo MTRANS.

Ademas, lo que se busca encontrar en los proyectos de data science son grupos especificos que puedan ayudar al modelo a identificar clases. Por esta razon, se plantean combinaciones de columnas para luego realizar One Hot Encoding. La utilidad de estas columnas asi como un Feature Selection, de ser necesario, sera evaluado en otros notebooks.

### Combinaciones propuestas:
    - Gender | Age (Bin)
    - CAEC | Age (Bin)
    - Gender | family_history_with_overweight
    - Gender | FAF(Bin)

In [15]:
data.head()

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,Female,21.0,1.62,64.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,0.0,1.0,no,Public_Transportation,Normal_Weight
1,Female,21.0,1.52,56.0,yes,no,3.0,3.0,Sometimes,yes,3.0,yes,3.0,0.0,Sometimes,Public_Transportation,Normal_Weight
2,Male,23.0,1.80,77.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,2.0,1.0,Frequently,Public_Transportation,Normal_Weight
3,Male,27.0,1.80,87.0,no,no,3.0,3.0,Sometimes,no,2.0,no,2.0,0.0,Frequently,Walking,Overweight_Level_I
4,Male,22.0,1.78,89.8,no,no,2.0,1.0,Sometimes,no,2.0,no,0.0,0.0,Sometimes,Public_Transportation,Overweight_Level_II


### Binning de Age y FAF

Los bins de edad de localizaran segun el analisis de la data. Los grupos mas importantes son de [15-20], [24-30] y [30-35]. Se realizaran particiones en 15, 20, 24, 30 y 35. Menores de 15 solo hay dos personas por lo que se adaptara al rango de [0-20]

In [42]:
cut_labels = [1, 2, 3, 4,5]
cut_bins = [0, 20, 24, 30,35,999]
data['Age_bin'] = pd.cut(data['Age'], bins=cut_bins, labels=cut_labels)

In [43]:
data['Age_bin'].value_counts()

2    702
1    585
3    468
4    188
5    168
Name: Age_bin, dtype: int64

In [44]:
data[data['Age_bin'].isnull()]

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad,Age_bin,FAF_bin


Los bins de FAF se localizaran segun el analisis de la data. Los grupos mas importantes son de [0-0.5], [0.5-1.5] y [1.5-2.5]. Se realizaran particiones en 0.5, 1.5, 2.5.

In [45]:
cut_labels = [1, 2, 3, 4]
cut_bins = [-1,0.5, 1.5, 2.5, 99999]
data['FAF_bin'] = pd.cut(data['FAF'], bins=cut_bins, labels=cut_labels)

In [46]:
data['FAF_bin'].value_counts()

2    776
1    720
3    496
4    119
Name: FAF_bin, dtype: int64

In [47]:
data[data['FAF_bin'].isnull()]

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad,Age_bin,FAF_bin


Se confirma que los bins han sido elaborados correctamente por lo que se pasa a la commbinación de columnas.

### Combinacion de columnas

In [51]:
data["Gender|Age"]=data["Gender"]+"_" +data["Age_bin"].astype(str)

In [53]:
data["CAEC|Age"]=data["CAEC"]+"_" +data["Age_bin"].astype(str)

In [54]:
data["Gender|FO"]=data["Gender"]+"_" +data["family_history_with_overweight"]

In [57]:
data["Gender|FAF"]=data["Gender"]+"_" +data["FAF_bin"].astype(str)

### Transformando columnas binarias,categoricas y ordinales

In [59]:
numeric=["Age","Height","Weight","FCVC","NCP","CH2O","FAF","TUE"]
ordinal=["CAEC","CALC"]
flags=["family_history_with_overweight","FAVC","SMOKE","SCC"]
categorical=["MTRANS"]

In [60]:
data["CAEC"]=data["CAEC"].map({"no":0,"Sometimes":1,"Frequently":2,"Always":3})

In [61]:
data["CALC"]=data["CALC"].map({"no":0,"Sometimes":1,"Frequently":2,"Always":3})

In [62]:
data["Gender"]=data["Gender"].map({"Female":0,"Male":1})

In [63]:
for column in flags:
    data[column]=data[column].map({"yes":1,"no":0})

In [68]:
data["MTRANS"] = data["MTRANS"].astype("category")

In [74]:
dictionary_of_columns_with_index_to_categorical = dict()
dictionary_of_columns_with_categorical_to_index = dict()
    
dictionary_of_columns_with_index_to_categorical["MTRANS"] = dict()
dictionary_of_columns_with_categorical_to_index["MTRANS"] = dict()

for index, category in enumerate(data["MTRANS"].cat.categories):
        dictionary_of_columns_with_index_to_categorical["MTRANS"][index] = category
        dictionary_of_columns_with_categorical_to_index["MTRANS"][category] = index
        
data.replace(dictionary_of_columns_with_categorical_to_index, inplace=True)

In [76]:
data["NObeyesdad"]=data["NObeyesdad"].map({"Insufficient_Weight":0,"Normal_Weight":1,"Overweight_Level_I":2,
                                           "Overweight_Level_II":3,"Obesity_Type_I":4,"Obesity_Type_II":5,
                                           "Obesity_Type_III":6})

### Se crea la columna BMI 

In [82]:
data["BMI"]=data["Weight"]/(data["Height"]*data["Height"])

### Ordenamiento de columnas y split de los datos

Se crearan dos versiones del dataset. La primera version es el dataset base procesando las columnas, y agregandole el BMI. La segunda version es la que utiliza Age_bin y FAF_bin y realiza feature engineering. Los dataset se pueden variar un poco para el entrenamiento de los modelos para mejorar su performance

#### Particion segunda version (Combinacion de columnas y binning)

In [83]:
data_2=pd.get_dummies(data,columns=['MTRANS', 'Gender|Age', 'CAEC|Age',
                           'Gender|FO','Gender|FAF'])

In [86]:
data_2.columns

Index(['Gender', 'Age', 'Height', 'Weight', 'family_history_with_overweight',
       'FAVC', 'FCVC', 'NCP', 'CAEC', 'SMOKE', 'CH2O', 'SCC', 'FAF', 'TUE',
       'CALC', 'NObeyesdad', 'Age_bin', 'FAF_bin', 'BMI', 'MTRANS_0',
       'MTRANS_1', 'MTRANS_2', 'MTRANS_3', 'MTRANS_4', 'Gender|Age_Female_1',
       'Gender|Age_Female_2', 'Gender|Age_Female_3', 'Gender|Age_Female_4',
       'Gender|Age_Female_5', 'Gender|Age_Male_1', 'Gender|Age_Male_2',
       'Gender|Age_Male_3', 'Gender|Age_Male_4', 'Gender|Age_Male_5',
       'CAEC|Age_Always_1', 'CAEC|Age_Always_2', 'CAEC|Age_Always_3',
       'CAEC|Age_Always_4', 'CAEC|Age_Always_5', 'CAEC|Age_Frequently_1',
       'CAEC|Age_Frequently_2', 'CAEC|Age_Frequently_3',
       'CAEC|Age_Frequently_4', 'CAEC|Age_Frequently_5',
       'CAEC|Age_Sometimes_1', 'CAEC|Age_Sometimes_2', 'CAEC|Age_Sometimes_3',
       'CAEC|Age_Sometimes_4', 'CAEC|Age_Sometimes_5', 'CAEC|Age_no_1',
       'CAEC|Age_no_2', 'CAEC|Age_no_3', 'CAEC|Age_no_4',
       'Gende

In [89]:
data_2=data_2[['Gender', 'Age', 'Height', 'Weight', 'BMI','family_history_with_overweight',
       'FAVC', 'FCVC', 'NCP', 'CAEC', 'SMOKE', 'CH2O', 'SCC', 'FAF', 'TUE',
       'CALC', 'NObeyesdad', 'Age_bin', 'FAF_bin','MTRANS_0',
       'MTRANS_1', 'MTRANS_2', 'MTRANS_3', 'MTRANS_4', 'Gender|Age_Female_1',
       'Gender|Age_Female_2', 'Gender|Age_Female_3', 'Gender|Age_Female_4',
       'Gender|Age_Female_5', 'Gender|Age_Male_1', 'Gender|Age_Male_2',
       'Gender|Age_Male_3', 'Gender|Age_Male_4', 'Gender|Age_Male_5',
       'CAEC|Age_Always_1', 'CAEC|Age_Always_2', 'CAEC|Age_Always_3',
       'CAEC|Age_Always_4', 'CAEC|Age_Always_5', 'CAEC|Age_Frequently_1',
       'CAEC|Age_Frequently_2', 'CAEC|Age_Frequently_3',
       'CAEC|Age_Frequently_4', 'CAEC|Age_Frequently_5',
       'CAEC|Age_Sometimes_1', 'CAEC|Age_Sometimes_2', 'CAEC|Age_Sometimes_3',
       'CAEC|Age_Sometimes_4', 'CAEC|Age_Sometimes_5', 'CAEC|Age_no_1',
       'CAEC|Age_no_2', 'CAEC|Age_no_3', 'CAEC|Age_no_4',
       'Gender|FO_Female_no', 'Gender|FO_Female_yes', 'Gender|FO_Male_no',
       'Gender|FO_Male_yes', 'Gender|FAF_Female_1', 'Gender|FAF_Female_2',
       'Gender|FAF_Female_3', 'Gender|FAF_Female_4', 'Gender|FAF_Male_1',
       'Gender|FAF_Male_2', 'Gender|FAF_Male_3', 'Gender|FAF_Male_4']]

In [90]:
data_2.head()

,Gender,Age,Height,Weight,BMI,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,NObeyesdad,Age_bin,FAF_bin,MTRANS_0,MTRANS_1,MTRANS_2,MTRANS_3,MTRANS_4,Gender|Age_Female_1,Gender|Age_Female_2,Gender|Age_Female_3,Gender|Age_Female_4,Gender|Age_Female_5,Gender|Age_Male_1,Gender|Age_Male_2,Gender|Age_Male_3,Gender|Age_Male_4,Gender|Age_Male_5,CAEC|Age_Always_1,CAEC|Age_Always_2,CAEC|Age_Always_3,CAEC|Age_Always_4,CAEC|Age_Always_5,CAEC|Age_Frequently_1,CAEC|Age_Frequently_2,CAEC|Age_Frequently_3,CAEC|Age_Frequently_4,CAEC|Age_Frequently_5,CAEC|Age_Sometimes_1,CAEC|Age_Sometimes_2,CAEC|Age_Sometimes_3,CAEC|Age_Sometimes_4,CAEC|Age_Sometimes_5,CAEC|Age_no_1,CAEC|Age_no_2,CAEC|Age_no_3,CAEC|Age_no_4,Gender|FO_Female_no,Gender|FO_Female_yes,Gender|FO_Male_no,Gender|FO_Male_yes,Gender|FAF_Female_1,Gender|FAF_Female_2,Gender|FAF_Female_3,Gender|FAF_Female_4,Gender|FAF_Male_1,Gender|FAF_Male_2,Gender|FAF_Male_3,Gender|FAF_Male_4
0,0,21.0,1.62,64.0,24.386526,1,0,2.0,3.0,1,0,2.0,0,0.0,1.0,0,1,2,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0
1,0,21.0,1.52,56.0,24.238227,1,0,3.0,3.0,1,1,3.0,1,3.0,0.0,1,1,2,4,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
2,1,23.0,1.80,77.0,23.765432,1,0,2.0,3.0,1,0,2.0,0,2.0,1.0,2,1,2,3,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0
3,1,27.0,1.80,87.0,26.851852,0,0,3.0,3.0,1,0,2.0,0,2.0,0.0,2,2,3,3,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0
4,1,22.0,1.78,89.8,28.342381,0,0,2.0,1.0,1,0,2.0,0,0.0,0.0,1,3,2,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0


In [98]:
x_2 = data_2.drop("NObeyesdad", axis=1)
y_2 = data_2["NObeyesdad"]

In [99]:
x_train_2, x_test_2, y_train_2, y_test_2 = train_test_split(x_2, y_2, test_size=0.2, random_state=42,stratify=y)

In [100]:
print("Tamaño de la data de entrenamiento:{}".format(len(x_train_2)))

Tamaño de la data de entrenamiento:1688


In [101]:
print("Tamaño de la data de entrenamiento:{}".format(len(y_test_2)))

Tamaño de la data de entrenamiento:423


In [102]:
x_train_2.head()

,Gender,Age,Height,Weight,BMI,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,Age_bin,FAF_bin,MTRANS_0,MTRANS_1,MTRANS_2,MTRANS_3,MTRANS_4,Gender|Age_Female_1,Gender|Age_Female_2,Gender|Age_Female_3,Gender|Age_Female_4,Gender|Age_Female_5,Gender|Age_Male_1,Gender|Age_Male_2,Gender|Age_Male_3,Gender|Age_Male_4,Gender|Age_Male_5,CAEC|Age_Always_1,CAEC|Age_Always_2,CAEC|Age_Always_3,CAEC|Age_Always_4,CAEC|Age_Always_5,CAEC|Age_Frequently_1,CAEC|Age_Frequently_2,CAEC|Age_Frequently_3,CAEC|Age_Frequently_4,CAEC|Age_Frequently_5,CAEC|Age_Sometimes_1,CAEC|Age_Sometimes_2,CAEC|Age_Sometimes_3,CAEC|Age_Sometimes_4,CAEC|Age_Sometimes_5,CAEC|Age_no_1,CAEC|Age_no_2,CAEC|Age_no_3,CAEC|Age_no_4,Gender|FO_Female_no,Gender|FO_Female_yes,Gender|FO_Male_no,Gender|FO_Male_yes,Gender|FAF_Female_1,Gender|FAF_Female_2,Gender|FAF_Female_3,Gender|FAF_Female_4,Gender|FAF_Male_1,Gender|FAF_Male_2,Gender|FAF_Male_3,Gender|FAF_Male_4
442,1,26.000000,1.720000,65.000000,21.971336,1,1,2.000000,3.000000,1,0,2.000000,0,0.000000,1.000000,1,3,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0
253,0,26.000000,1.590000,47.000000,18.591037,1,1,2.000000,1.000000,1,0,2.000000,0,0.000000,2.000000,1,3,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0
554,1,16.198153,1.691007,52.629374,18.405067,0,1,2.000000,2.000986,1,0,2.673835,0,0.992950,0.474836,1,1,2,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0
1500,1,24.481032,1.661277,90.744965,32.880501,1,1,1.712848,3.000000,1,0,1.162153,0,0.458981,0.885532,0,3,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0
359,0,33.000000,1.560000,48.000000,19.723866,1,0,2.000000,3.000000,1,0,2.000000,0,1.000000,0.000000,1,4,2,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0


In [103]:
x_train_2.to_csv("X_train_v2.csv",index=False)
y_train_2.to_csv("Y_train_v2.csv",index=False)
x_test_2.to_csv("X_test_v2.csv",index=False)
y_test_2.to_csv("Y_test_v2.csv",index=False)

#### Particion primera version (Solo BMI extra)

In [85]:
data.columns

Index(['Gender', 'Age', 'Height', 'Weight', 'family_history_with_overweight',
       'FAVC', 'FCVC', 'NCP', 'CAEC', 'SMOKE', 'CH2O', 'SCC', 'FAF', 'TUE',
       'CALC', 'MTRANS', 'NObeyesdad', 'Age_bin', 'FAF_bin', 'Gender|Age',
       'CAEC|Age', 'Gender|FO', 'Gender|FAF', 'BMI'],
      dtype='object')

In [87]:
data=data[['Gender', 'Age', 'Height', 'Weight','BMI', 'family_history_with_overweight',
       'FAVC', 'FCVC', 'NCP', 'CAEC', 'SMOKE', 'CH2O', 'SCC', 'FAF', 'TUE',
       'CALC', 'MTRANS', 'NObeyesdad']]

In [88]:
data.head()

,Gender,Age,Height,Weight,BMI,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,0,21.0,1.62,64.0,24.386526,1,0,2.0,3.0,1,0,2.0,0,0.0,1.0,0,3,1
1,0,21.0,1.52,56.0,24.238227,1,0,3.0,3.0,1,1,3.0,1,3.0,0.0,1,3,1
2,1,23.0,1.80,77.0,23.765432,1,0,2.0,3.0,1,0,2.0,0,2.0,1.0,2,3,1
3,1,27.0,1.80,87.0,26.851852,0,0,3.0,3.0,1,0,2.0,0,2.0,0.0,2,4,2
4,1,22.0,1.78,89.8,28.342381,0,0,2.0,1.0,1,0,2.0,0,0.0,0.0,1,3,3


In [91]:
x = data.drop("NObeyesdad", axis=1)
y = data["NObeyesdad"]

In [92]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42,stratify=y)

In [95]:
print("Tamaño de la data de entrenamiento:{}".format(len(x_train)))

Tamaño de la data de entrenamiento:1688


In [97]:
print("Tamaño de la data de entrenamiento:{}".format(len(y_test)))

Tamaño de la data de entrenamiento:423


In [104]:
x_train.head()

,Gender,Age,Height,Weight,BMI,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS
442,1,26.000000,1.720000,65.000000,21.971336,1,1,2.000000,3.000000,1,0,2.000000,0,0.000000,1.000000,1,4
253,0,26.000000,1.590000,47.000000,18.591037,1,1,2.000000,1.000000,1,0,2.000000,0,0.000000,2.000000,1,3
554,1,16.198153,1.691007,52.629374,18.405067,0,1,2.000000,2.000986,1,0,2.673835,0,0.992950,0.474836,1,3
1500,1,24.481032,1.661277,90.744965,32.880501,1,1,1.712848,3.000000,1,0,1.162153,0,0.458981,0.885532,0,0
359,0,33.000000,1.560000,48.000000,19.723866,1,0,2.000000,3.000000,1,0,2.000000,0,1.000000,0.000000,1,3


In [105]:
x_train.to_csv("X_train.csv",index=False)
y_train.to_csv("Y_train.csv",index=False)
x_test.to_csv("X_test.csv",index=False)
y_test.to_csv("Y_test.csv",index=False)